내용 : 피자 주문을 받는 챗봇을 만들고 대화하기 

주요점 : .append를 사용하여 대화내용을 저장하도록 만들기

※ 주의점 : 현재 이 코드는 무조건 get_menu(메뉴불러오기)를 실행한 후에야 동작함

In [20]:
import json
from dotenv import load_dotenv
import gradio as gr
import openai
import requests
import inspect # 함수의 형식을 알아내기 위해 사용
import uuid   # 임의 문자열 생성
load_dotenv()
client = openai.OpenAI()

In [21]:
menu = {
    "슈퍼 슈프림": 10000,
    "아이리쉬 포테이토": 11000,
    "스파이스 치킨": 12000,
    "슈프림 알프레도": 13000
}

size_price = {"레귤러": 0, "미디엄":2000, "라지": 5000, "패밀리": 10000}

In [22]:
# 딕셔너리 형태로 {메뉴, 사이즈} 넣을 수 있도록 미리 구조 설계
def get_menu() -> dict:
    return {"menu":menu, "size":size_price}

In [23]:
# # 가격 계산
# # 인자값은 각각 (메뉴명, 사이즈, 수량)을 딕셔너리 타입으로
# # pizza_name, pizza_size, quantity가 어디서 정해지는지 보려면 myfunctions 로
# def get_order_price(pizza_name:str, pizza_size:str="레귤러",quantity:int=1)->dict:
#     base = menu.get(pizza_name,0)    # 없는 피자이름이면 0
#     extra = size_price.get(pizza_size, 0)   
#     unit_price = base + extra 
#     sub_total = unit_price * quantity
#     return {
#         "pizza_name": pizza_name,
#         "pizza_size": pizza_size,
#         "quantity": quantity,
#         "unit_price": unit_price,    # 기본가격 + 사이즈 + (토핑)
#         "sub_total": sub_total     # x 수량
#     }

In [24]:
# 구매 확정 : orders 는 주문 내역을 저장하는 딕셔너리
def set_order(pizza_name:str, pizza_size:str="레귤러",quantity:int=1, orders:dict=None)->dict:
    if orders is None:  # 추가주문이면 None 이 아닙니다.
        orders = {"order_id": None, "content":[], "payment":0} # 초기값 세팅

    base = menu.get(pizza_name,0)    # 없는 피자이름이면 0
    extra = size_price.get(pizza_size, 0)   
    unit_price = base + extra
    sub_total = unit_price * quantity

    if not orders.get("order_id"):    # 처음 주문이면 None
        orders["order_id"] = str(uuid.uuid4())   # 주문번호 난수 문자열로 생성

    orders['content'].append({
        "order": {"pizza_name": pizza_name,"pizza_size": pizza_size, "quantity": quantity },
        "unit_price": unit_price,    # 기본가격 + 사이즈 + (토핑)
        "sub_total": sub_total 
        })

    orders['payment'] = sum(item['sub_total'] for item in orders['content'])
    return {
        "pizza_name": pizza_name,
        "pizza_size": pizza_size,
        "quantity": quantity,
        "unit_price": unit_price,    # 기본가격 + 사이즈 + (토핑)
        "sub_total": sub_total     # x 수량
    }

In [25]:
# GPT에게 먹일 프롬프트
instruction = """
너는 피자 주문을 돕는 챗봇이다.
친절하고 간단명료하게 대화하며, 고객이 원하는 피자를 정확히 주문할 수 있도록 안내한다.
1. 고객의 주문 의도를 파악한다 - 주문 내용은 메뉴 이름, 사이즈, 수량이며 그외에는 없음.
주의 : 메뉴 목록에 없는 것은 주문 받으면 안됨.
2. 피자의 종류와 사이즈는 지시한 대로만 가격 안내 해야 함.
3. 주문을 진행하면 총 결제금액을 계산하여 알려주고 주문 번호를 생성하여 저장한다.
4. 추가 주문이 없는지 확인하고 주문을 확정하면 주문번호와 주문 내역, 결제금액을 확인한다.
5. 불필요한 잡담은 최소화하고, 주문과 관련된 대화에 집중한다.
6. 항상 정중하고 친근한 말투를 유지한다.
가벼운 인사말과 피자 주문 이외의 다른 요청은 '챗봇 기능과 다른 질문입니다.' 라고 답변해.
"""

In [26]:
myfunctions = [
    {
        "name": "get_menu",
        "description": "피자 메뉴 목록을 조회합니다.",
        "parameters": {}
    },
    {
        "name": "get_order_price",
        "description": "사용자의 문의에 따라 피자 주문을 위한 가격을 계산합니다..",
        "parameters": {
            "type": "object",
            "properties": {
                "pizza_name": {"type": "string"},
                "pizza_size": {"type": "string"},
                "quantity": {"type": "integer"},
            },
            "required": ["pizza_name", "pizza_size", "quantity"]
        }
    },
    {
        "name": "set_order",
        "description": "피자 주문을 생성하거나 추가합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "pizza_name": {"type": "string"},
                "pizza_size": {"type": "string"},
                "quantity": {"type": "integer"},
                "orders":{"type":"object"}
            },
            "required": ["pizza_name", "pizza_size", "quantity","orders"]
        }
    },
    {
        "name": "complete_order",
        "description": "주문을 완료하고 결제 정보를 반환합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "orders":{"type":"object"}
            },
            "required": ["orders"]
        }
    }
]

In [27]:
def get_first_chatbot_response(chat_history):
  response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=chat_history,
    functions=myfunctions,
    function_call="auto"  
  )
  return response

In [28]:
def get_followup_chatbot_response(chat_history):
    followup_response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=chat_history,
        functions=myfunctions
    )
    return followup_response

In [ ]:
def chat_with_bot(user_input,chat_history,orders):
    if not chat_history :
        chat_history.append({"role":"system", "content":instruction}) # 위의 프롬프트 먹이기
    chat_history.append({"role": "user", "content": user_input}) # 입력값 받기
    response = get_first_chatbot_response(chat_history) # gpt에게 어떤 함수 사용할지 응답받기
    print(f'log resp : {response}')
    message = response.choices[0].message
    print(f'log  function_call\n↪: {message.function_call}')
    if message.function_call:
        fn_name = message.function_call.name # 함수명 저장
        # args = json.loads(message["function_call"]["arguments"])
        args = json.loads(message.function_call.arguments) # 인자를 dict 타입으로 변경함.
        print(f'log args\n↪: {args}')
        result = globals()[fn_name](**args)   # 지정한 함수 이름으로 실제 함수 가져와 실행하기
        print(f'log result\n↪: {result}')
        # chat_history.append(message)
        chat_history.append({
            "role": "function",
            "name": fn_name,
            "content": json.dumps(result, ensure_ascii=False),
        })

        final_response =get_followup_chatbot_response(chat_history)
        reply = final_response.choices[0].message.content
    else:
        reply = message.content
    
    # 사용자에게 보낸 응답은 assistant role
    chat_history.append({"role": "assistant", "content": reply})
    return '', chat_history,orders

In [30]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🍕 피자 주문 챗봇")

    chat_output = gr.Chatbot(label="PizzaOrderBot",type="messages")
    user_input = gr.Textbox(placeholder="예: 슈퍼 슈프림 주문, 주문 완료", label="메시지 입력")
    submit = gr.Button("전송")

    # State 컴포넌트 : 앱 실행 중에 값을 유지하기 위해서 사용. 보이지는 않는 컴포넌트
    # 대화 내용 기록
    chat_history = gr.State([])   # [] 는 초기값
    # 주문 내용 기록
    orders = gr.State({"order_id": None, "content": [], "payment": 0})

     # 전송 버튼
    submit.click(
        chat_with_bot,
        inputs=[user_input, chat_history, orders], # orders 를 chat_with_bot 전달
        outputs=[user_input, chat_output, orders], # chat_with_bot 함수에 변경된 orders 리턴
    )
   # 입력창에서 엔터로 실행 
    user_input.submit(
        chat_with_bot,
        inputs=[user_input, chat_history, orders],
        outputs=[user_input, chat_output, orders],
    )

demo.launch(inline=False,debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
log resp : ChatCompletion(id='chatcmpl-CbjXggpBkKmy19F0r1QajsdgMT38e', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{}', name='get_menu'), tool_calls=None))], created=1763109236, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_4c2851f862', usage=CompletionUsage(completion_tokens=10, prompt_tokens=363, total_tokens=373, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
log  function_call
↪: FunctionCall(arguments='{}', name='get_menu')
log args
↪: {}
log result
↪: {'menu': {'슈퍼 슈프림': 10000,

Traceback (most recent call last):
  File "c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\gradio\blocks.py", line 2127, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\gradio\blocks.py", line 1904, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\C118\miniconda3\envs\llm

log resp : ChatCompletion(id='chatcmpl-CbjY2FH88FdPYlVFhrIPTGRkrTOq4', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{}', name='get_menu'), tool_calls=None))], created=1763109258, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_4c2851f862', usage=CompletionUsage(completion_tokens=10, prompt_tokens=363, total_tokens=373, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
log  function_call
↪: FunctionCall(arguments='{}', name='get_menu')
log args
↪: {}
log result
↪: {'menu': {'슈퍼 슈프림': 10000, '아이리쉬 포테이토': 11000, '스파이스 치킨': 12000, '슈프림 알프레도': 13000}, 'size': {'레귤러': 0, '미디엄': 2000, '라지': 5000, '패밀

Traceback (most recent call last):
  File "c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\gradio\blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    r

Keyboard interruption in main thread... closing server.


In [31]:
demo.close()

Closing server running on port: 7860
